In [58]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr

## Black-Scholes related utilities

In [59]:
def norm_cdf(x):
    """ Approximation of the normal distribution function with an error less than 7.5*10^-8 """
    assert x >= 0 or x < 0
    coefficients = [0.2316419, 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.33027442]
    if x >= 0:
        t = 1 / (1 + coefficients[0] * x)
        terms = [coefficients[i] * t**i for i in range(1, 6)]
        approx = 1 - (1 / np.sqrt(2 * np.pi)) * np.exp(-x**2 / 2) * sum(terms)
        return approx
    else:
        return 1 - norm_cdf(-x)

In [60]:
def call_eur_bs(r, sig, T, S0, K):
    d = (np.log(S0 / K) + r * T + 0.5 * sig**2 * T) / (sig * np.sqrt(T))
    return (
        S0 * np.exp(r * T) * norm_cdf(sig * np.sqrt(T) - d)
        - K * norm_cdf(-d)
    )

In [61]:
def price_paths_bs(r, sig, T, S0, num_paths, num_subdivs):
    step = (T/num_subdivs)
    log_diffs = (r-0.5*sig**2) * step + sig*np.sqrt(step)*npr.normal(0, 1, size=(num_paths, num_subdivs))
    log_diffs = np.concatenate((np.log(S0)*np.ones((num_paths, 1)), log_diffs), axis=1)
    log_paths = np.cumsum(log_diffs, axis=1)
    price_paths = np.exp(log_paths)

    return price_paths

## Turnbull & Wakeman approximation

Turnbull & Wakeman have provided an approximation of the price of an asian option, as a european option with different parameters.

In [62]:
def tnw_coeffs_con(r, sig, T):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    M1 = (np.exp(r*T)-1)/(r*T)
    M2 = ( 
        (2*np.exp((2*r+sig**2)*T)) / ((r+sig**2) * (2*r+sig**2) * T**2)
        + (2/(r*T**2)) * (1/(2*r+sig**2) - np.exp(r*T) / (r+sig**2))
    )

    r_a = np.log(M1) / T 
    sig_a = np.sqrt(-2*r_a + np.log(M2)/T)
    return (r_a, sig_a)

In [63]:
def tnw_coeffs_dis(r, sig, T, num_subdivs):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    dt = T/num_subdivs
    N = num_subdivs
    t0 = np.exp((2*r+sig**2)*dt)
    t1 = np.exp(r*dt)
    M1 = (1 / N)*( (t1*(1-np.exp(r*T))) / (1-t1) )
    M2 = (
        (1/(N**2))*( (t0 * (1-np.exp((2*r+sig**2)*T)) ) / (1-t0) )
        + (1/(N**2))*(2*t1/(1-t1))*( 
        t0 * ( (1-np.exp((2*r+sig**2)*(N-1)*dt)) / (1-t0) )
        - np.exp(((N+1)*r+sig**2)*dt) * ( (1-np.exp((r+sig**2)*(N-1)*dt)) / (1-np.exp((r+sig**2)*dt)) )
    ))

    r_A = np.log(M1) / T
    sig_a = np.sqrt(np.log(M2)/T - 2 * r_A)

    return (r_A, sig_a)

In [64]:
def call_asian_bs_tnw(r, sig, T, S0, K, num_subdivs):
    """ Calculates price of an asian option under the Turnbull & Wakeman approximation and the Black-Scholes model """
    r_a, sig_a = tnw_coeffs_dis(r, sig, T, num_subdivs)
    return np.exp(-r*T) * call_eur_bs(r_a, sig_a, T, K, S0)

## Monte-Carlo approximation

In [65]:
def call_asian_bs_mc(r, sig, T, S0, K, num_paths, num_subdivs):
    price_paths = price_paths_bs(r, sig, T, S0, num_paths, num_subdivs)

    payoffs_asian = np.exp(-r*T)*np.maximum(price_paths.mean(axis=1) - K, 0)
    var = payoffs_asian.var(ddof=1) / num_paths

    return payoffs_asian.mean(), var

In [155]:
def ctrl_coeffs(r, sig, n):
    sig_e = (sig/n)*np.sqrt( ((n+1)*(2*n+1)) / 6 )
    r_e = 0.5*sig_e**2 + (r-0.5*sig**2)*(n+1)/(2*n)

    return (r_e, sig_e)

In [182]:
def call_asian_bs_mc_ctrl(r, sig, T, S0, K, num_paths, num_subdivs):
    """ Calculate a monte-carlo simulation of the asian price with a lower variance """

    price_paths = price_paths_bs(r, sig, T, S0, num_paths, num_subdivs)
    payoffs_asian = np.maximum(price_paths.mean(axis=1) - K, 0)
    control_samples = np.maximum(np.exp( np.log(price_paths)[:,1:].mean(axis=1) ) - K, 0)

    r_e, sig_e = ctrl_coeffs(r, sig, num_subdivs)
    control_mean = call_eur_bs(r_e, sig_e, T, S0, K)
    res_sample = np.exp(-r*T)*(payoffs_asian - control_samples + control_mean)

    print(control_samples.mean(), control_mean)

    var = res_sample.var(ddof=1) / num_paths
    return (res_sample.mean(), var)
    

## Tests

In [131]:
# Spot
S0 = 100
# Strike
K = 100
# Vol.
sig = 0.3
# Expiration
T = 6
# Risk-free rate
r = 0.02
# Number of simulations (monte-carlo)
N = 10000
# Path subdivisions
n = 20

In [186]:
tnw_approx = call_asian_bs_tnw(r, sig, T, S0, K, num_subdivs=n)
mc_approx, var = call_asian_bs_mc(r, sig, T, S0, K, num_paths=N, num_subdivs=n)
mc_approx_c, var_c = call_asian_bs_mc_ctrl(r, sig, T, S0, K, num_paths=N, num_subdivs=n)

print(f"Approximation Monte-Carlo  : {mc_approx:.3f}€ (var={var:.6f})")
print(f"Variable de contrôle       : {mc_approx_c:.3f}€ (var={var_c:.6f})")
print(f"Approximation T&W discrète : {tnw_approx:.3f}€")

18.347112850611804 18.447549329349705
Approximation Monte-Carlo  : 17.944€ (var=0.118423)
Variable de contrôle       : 18.072€ (var=0.001954)
Approximation T&W discrète : 18.948€
